# Challenge: Hyperbolic Embedding via Graph Learning
Authors: Rishi Sonthalia and Xinyue Cui

## 1 Introduction

Many different types of datasets are better represented in hyperbolic space as compared to Euclidean space. These are normally datasets that have semantically rich hierarchies such as text [35], social and other forms of networks [7,9,25,26], and cell development trees [16]. Other applications of hyperbolic representations have also been seen to be useful for computer vision and for other tasks [32,33,34]. Based off of the success of these applications, recently there has been great effort in developing hyperbolic versions of different neural networks - hyperbolic deep neural networks [4,39,46,47], hyperbolic convolutional neural networks [47,48], hyperbolic recurrent neural networks [39], hyperbolic transformers [40,47], and hyperbolic graph neural networks [42,43,45]. See survey [49] for many for example applications.  

Additionally, people have looked at extending hyperbolic embeddings beyong hyperbolic manifolds to other hyperbolic structures such as Hierarchical Hyperbolic Spaces [38]. Work such as [36,37] have proven that phylogentic data live in these Hierarchical Hyperbolic Spaces.


To take advantage of hyperbolic representations we need techniques that learn embed data into hyperbolic manifolds. Historiocally two different approaches have been taken to learning these embeddings. First, we learn these embeddings by solving an optimization problem. This could be done in a supervised on an unsupervised manner. Second, we learn combinatorial structures that are then combinatorially embedded into the hyperbolic manifold.

### 1.1 Optimiation Based Methods. 

For unsupervised method, we are given a distance matrix or a distance matrix is extracted from the given data and then we set up an optimization problem that learns the embedding into the hyperbolic manifold. Concretely, let $D \in \mathbb{R}^{n \times n}$ be the distance matrix, let $y_i$ for $i = 1, \ldots, n$ be the vectors in the hyperbolic manifold $\mathbb{H}^d$. Then we have a loss function $L := L(D, y_1, \ldots, y_n)$ that we want to minimize. Examples of such methods include [2,4,17,18,41]. 

Supervised methods are methods that learn embeddings using neural networks. One common example is to learn work embeddings, works such as [50,51] present hyperbolic versions of standard word embedding techniques such as Word2Vec and GLoVe. Another approach to pick a random vector in $\mathbb{H}^d$ as the initial embedding and this embedding is optimized during the training of the neural network. 

However, such approaches tend to have a few common issues. 

    1) The optimization problem is non convex and is very difficult to solve;
    2) the otpimiation procedure is unstable and require large number of bits to accruately represent; or
    3) the methods ignore the geometry of the input data;
    4) the methods do not have any theoretical gaurantees. 
    
### 1.2 Combinatorial Techniques. 

There also many combinatorial methods. These certain around two different technqiues. The first is to directly embed a given graph into a hyperbolic manifold []. Other methods exploit the tree like structure [] of hyperbolic space. For these methods, given a metric, we first learn a tree that approximates the given metric []. Then given this tree we either treat the tree as the hyperbolic representation or embed the tree in a hyperbolic manifold using Sarkars algorithms and its extensions []. 

### 1.3 This Notebook

In this notebook, we would like to implement one such combinatorial method. Specifically, we present the latest tree learning method [] as well as Sarkars algorithm []. The structure of the notebook will be as follows

     1) First we present some background on hyperbolic geometry from both the differential and algebraic view point
     2) We present TreeRep and demonstrate how it can be used to construct trees from metrics
     3) We present Sarkars algorithm and demonstrate how it can be used to embed data into hyperbolic disk. 
     4) We test the complete pipeline on real world and synthetic datasets to demonstrate that we get embeddings with low distortion. 
 
Other implementations of the above algorithms exist. TreeRep has two prior implementations. The first is in Julia and can be found at [] and the second in C++ and can be found at []. There are two prior implementations of Sarkars algorithm as well. Again there are in Julia [] and in C++ []. Both implementations for both algorithms have python wrappers. But to the best of our knowledge there does not exist a python version of the two algorithms. 


### 1.4 Related work

There is another family of work related to these methods. That is idea of approximating general metrics by trees. Note here we do not have hyperbolic view of these metrics or of the trees just that trees are simple graphical structures. Such work includes []

The final area of related work, is this the notion of metric embeddings and includes work such as []. 

# 3 Code 

In [ ]:
# Import the necessary packages

import networkx as nx
import geomstats.backend as gs

In [2]:
import TreeRep

INFO: Using numpy backend


## 2.1 TreeRep - Reconstructing Trees. 

## Synthetic Tests

1) Recovering Known Trees
2) Learing trees of random graphs
3) Learing trees for Random points in hyperbolic space

Embedding trees

1) Show scaling effect of embedding
2) Take random points in hyperbolic space learn tree and embed and then compare the points

## Applications 

1) Pure embeddings, do the karate club graph
2) Dimensionality reduction - take high dimensional data, learn tree, and then embed into two dimensional space
3) Learn Tree structure, such as a phylogentic tree.

In [9]:
from importlib import reload 
reload(TreeRep)

for trial in range(100,5000,100):
  n = 200
  G = nx.gnp_random_graph(200, 0.7)
  for e in G.edges():
      G[e[0]][e[1]]['weight'] = gs.random.rand()*10
  d = nx.algorithms.shortest_paths.dense.floyd_warshall(G)
  D = gs.zeros((n,n))
  for i in range(n):
    for j in range(n):
      D[i,j] = d[i][j]

  T = TreeRep.TreeRep(D)
  T.learn_tree()
  print(G.number_of_nodes(), G.number_of_edges())
  print(T.G.number_of_nodes(),T.G.number_of_edges())
  print(nx.is_k_edge_connected(T.G,1), nx.is_k_edge_connected(T.G,2))
  print()

200 13870
289 288
True False

200 13941
271 270
True False

200 13917
273 272
True False

200 13873
297 296
True False



KeyboardInterrupt: ignored

## 3.2 Sarkars Algorithm

## 3.3 Testing on Synthetic test

### 3.3.1 Random Points on the hyperbolic manifold

### 3.3.2 Random Sparse Graphs

## 3.4 Testing it on real world data

# 4 Bibliography

[1] Thomas Blasius, Tobias Friedrich, Anton Krohmer, Soren Laue, Anton Krohmer, Soren Laue, Tobias Friedrich, and Thomas Blasius. 2018. Efficient Embedding of Scale-Free Graphs in the Hyperbolic Plane. IEEE/ACM Trans. Netw. 26, 2 (April 2018), 920–933. DOI:https://doi.org/10.1109/TNET.2018.2810186 
 
 [2] Cvetkovski, Andrej and Crovella, Mark (2016) "Multidimensional Scaling in the Poincaré disk," Applied Mathematics & Information Sciences: Vol. 10 : Iss. 1 , Article 12. 
 DOI: http://dx.doi.org/10.18576/amis/100112 
 Available at: https://dc.naturalspublishing.com/amis/vol10/iss1/12 
 
 [3] Kevin Verbeek and Subhash Suri. 2014. Metric Embedding, Hyperbolic Space, and Social Networks. In Proceedings of the thirtieth annual symposium on Computational geometry (SOCG'14). Association for Computing Machinery, New York, NY, USA, 501–510. DOI:https://doi.org/10.1145/2582112.2582139 
 
 [4] Jörg A Walter, H-MDS: a new approach for interactive visualization with multidimensional scaling in the hyperbolic space, Information Systems, Volume 29, Issue 4, 2004, Pages 273-292, ISSN 0306-4379, https://doi.org/10.1016/j.is.2003.10.002. 
 
 [5] R. Kleinberg. 2007. Geographic Routing Using Hyperbolic Space. In Proceedings of the IEEE INFOCOM 2007 - 26th IEEE International Conference on Computer Communications. IEEE Computer Society, USA, 1902–1909. DOI:https://doi.org/10.1109/INFCOM.2007.221 
 
 [6] R. Krauthgamer and J. R. Lee, "Algorithms on negatively curved spaces," 2006 47th Annual IEEE Symposium on Foundations of Computer Science (FOCS'06), 2006, pp. 119-132, doi: 10.1109/FOCS.2006.9. 
 
 [7] Krioukov D, Papadopoulos F, Vahdat A, Boguñá M. Curvature and temperature of complex networks. Phys Rev E Stat Nonlin Soft Matter Phys. 2009 Sep;80(3 Pt 2):035101. doi: 10.1103/PhysRevE.80.035101. Epub 2009 Sep 23. PMID: 19905164. 
 
 [8] A. Cvetkovski and M. Crovella, "Hyperbolic Embedding and Routing for Dynamic Graphs," IEEE INFOCOM 2009, 2009, pp. 1647-1655, doi: 10.1109/INFCOM.2009.5062083. 
 
 [9] Y. Shavitt and T. Tankel, "Hyperbolic Embedding of Internet Graph for Distance Estimation and Overlay Construction," in IEEE/ACM Transactions on Networking, vol. 16, no. 1, pp. 25-36, Feb. 2008, doi: 10.1109/TNET.2007.899021. 
 
 [10] Matthias Hamann. On the Tree-Likeness of Hyperbolic Spaces. Mathematical Proceedings of the Cambridge Philosophical Society, 164(2):345–361, 2018. doi: 10.1017/ S0305004117000238. 
 
 [11] Anna Dyubina and Iosif Polterovich. Explicit Constructions of Universal R-Trees and Asymptotic Geometry of Hyperbolic Spaces. Bulletin of the London Mathematical Society, 33 (6):727?734, Nov 2001. 
 
 [12] M.Bonk and O.Schramm. Embeddings of Gromov Hyperbolic Spaces. Geometric & Functional Analysis GAFA, 10(2):266–306, Jun 2000. 
 
 [13] Ittai Abraham, Mahesh Balakrishnan, Fabian Kuhn, Dahlia Malkhi, Venugopalan Ramasubramanian, and Kunal Talwar. Reconstructing Approximate Tree Metrics. In Proceedings of the Twenty-sixth Annual ACM Symposium on Principles of Distributed Computing, PODC ’07, pages 43–52, New York, NY, USA, 2007. ACM. 
 
 [14] M.R. Bridson and A. Häfliger. Metric Spaces of Non-Positive Curvature. Grundlehren der mathematischen Wissenschaften. Springer Berlin Heidelberg, 2013. ISBN 9783662124949. 
 
 [15] Victor Chepoi, Feodor Dragan, Bertrand Estellon, Michel Habib, and Yann Vaxès. Diameters, Centers, and Approximating Trees of δ-Hyperbolic Geodesic Spaces and Graphs. In Proceedings of the Twenty-fourth Annual Symposium on Computational Geometry, SCG ’08, pages 59–68, New York, NY, USA, 2008. ACM. 
 
 [16] Anna Klimovskaia, David Lopez-Paz, Léon Bottou, and Maximilian Nickel. Poincaré Maps for Analyzing complex Hierarchies in Single-Cell Data. bioRxiv, 2019. doi: 10.1101/689547. 
 
 [17] Maximilian Nickel and Douwe Kiela. Poincaré Embeddings for Learning Hierarchical Representations. In NIPS, 2017. 
 
 [18] Frederic Sala, Chris De Sa, Albert Gu, and Christopher Re. Representation Tradeoffs for Hy- perbolic Embeddings. Proceedings of the 35th International Conference on Machine Learning, pages 4460–4469, July 2018. 
 
 [19] Rik Sarkar. Low Distortion Delaunay Embedding of Trees in Hyperbolic Plane. In Proceedings of the 19th International Conference on Graph Drawing, GD’11, pages 355–366, Berlin, Heidelberg, 2012. Springer-Verlag. 
 
 [20] B. P. Chamberlin, J. Clough, and M. P. Deisenroth, "Neural embeddings of graphs in hyperbolic space", arXiv 2017
 
 [21] M. E. Newman, “Power laws, pareto distributions and zipf’s law,” Contemporary physics, 2005.

 [22] H. W. Lin and M. Tegmark, “Critical behavior in physics and probabilistic formal languages,” Entropy, 2017.

 [23] K. Katayama and E. W. Maina, “Indexing method for hierarchical graphs based on relation among interlacing sequences of eigenvalues,” Journal of information processing, 2015.

 [24] R. Shimizu, Y. Mukuta, and T. Harada, “Hyperbolic neural networks++,” 2021.
 
 [25] M. Boguna ́, F. Papadopoulos, and D. Krioukov, “Sustaining the internet with hyperbolic mapping,” Nature communications, 2010.

 [26] B. Tadic ́, M. Andjelkovic ́, and M. Sˇuvakov, “Origin of hyperbolicity in brain-to-brain coordination networks,” Frontiers in Physics, 2018.
 
 [27] G.Garcia-Pe ́rez,M.Bogun ̃a ́,A.Allard,andM.A ́.Serrano,“Thehidden hyperbolic geometry of international trade: World trade atlas 1870– 2013,” Scientific reports, 2016.

 [28] M. Keller-Ressel and S. Nargang, “The hyperbolic geometry of financial networks,” Scientific reports, 2021.
 
 [29] R. Krauthgamer and J. R. Lee, “Algorithms on negatively curved spaces,” in 2006 47th Annual IEEE Symposium on Foundations of Computer Science (FOCS’06). IEEE, 2006.

 [30] E. Begelfor and M. Werman, “The world is not always flat or learning curved manifolds,” School of Engineering and Computer Science, Hebrew University of Jerusalem., Tech. Rep, 2005.

 [31] M. M. Bronstein, J. Bruna, Y. LeCun, A. Szlam, and P. Vandergheynst, “Geometric deep learning: going beyond euclidean data,” IEEE Signal Processing Magazine, 2017.
 
 [32] B. P. Chamberlain, S. R. Hardwick, D. R. Wardrope, F. Dzogang, F. Daolio, and S. Vargas, “Scalable hyperbolic recommender systems,” CoRR, 2019.

 [33] P. Kolyvakis, A. Kalousis, and D. Kiritsis, “Hyperkg: hyperbolic knowledge graph embeddings for knowledge base completion,” arXiv, 2019.

 [34] V. Khrulkov, L. Mirvakhabova, E. Ustinova, I. Oseledets, and V. Lempit- sky, “Hyperbolic image embeddings,” in Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition, 2020.
 
 [35] Bhuwan Dhingra, Christopher J. Shallue, Mohammad Norouzi, Andrew M. Dai, and George E. Dahl. Embedding Text in Hyperbolic Spaces, 2018.
 
 [36] Louis J. Billera, Susan P. Holmes, and Karen Vogtmann. Geometry of the space of phylogenetic trees. Advances in Applied Mathematics, 27(4):733–767, 2001. ISSN 0196-8858. doi: https:// doi.org/10.1006/aama.2001.0759. URL https://www.sciencedirect.com/science/ article/pii/S0196885801907596.
 
 [37] Katherine St John. Review paper: The shape of phylogenetic treespace. Systematic Biology, 66:e83 – e94, 2017.
 
 [38] CubeRep: Learning Relations Between Different Views of Data. 
 
 [39] O. Ganea, G. Be ́cigneul, and T. Hofmann, “Hyperbolic neural networks,” Advances in neural information processing systems, 2018.

 [40] C. Gulcehre, M. Denil, M. Malinowski, A. Razavi, R. Pascanu, K. M. Hermann, P. Battaglia, V. Bapst, D. Raposo, A. Santoro et al., “Hyperbolic attention networks,” arXiv, 2018.

 [41] M. Nickel and D. Kiela, “Learning continuous hierarchies in the lorentz model of hyperbolic geometry,” Proceedings of the 35-th International Conference on Machine Learning, PMLR, 2018.

 [42] Q. Liu, M. Nickel, and D. Kiela, “Hyperbolic graph neural networks,” in Advances in Neural Information Processing Systems, 2019.

 [43] I. Chami, Z. Ying, C. Re ́, and J. Leskovec, “Hyperbolic graph convolutional neural networks,” in Advances in neural information processing systems, 2019.

 [44] W. Peng, J. Shi, Z. Xia, and G. Zhao, “Mix dimension in poincare ́ geometry for 3d skeleton-based action recognition,” in Proceedings of the 28th ACM International Conference on Multimedia, 2020.

 [45] G. Bachmann, G. Be ́cigneul, and O. Ganea, “Constant curvature graph convolutional networks,” in International Conference on Machine Learning. PMLR, 2020.

 [46] D. J. Rezende, G. Papamakarios, S. Racaniere, M. S. Albergo, G. Kanwar, P. E. Shanahan, and K. Cranmer, “Normalizing flows on tori and spheres,” Proceedings of the 37th International Conference on Machine Learning, 2020.
 
 [47] Chen, W., Han, X., Lin, Y., Zhao, H., Liu, Z., Li, P., Sun, M., & Zhou, J. (2021). Fully Hyperbolic Neural Networks. ArXiv, abs/2105.14686.
 
 [48] Lensink, K., Haber, E., & Peters, B. (2019). Fully Hyperbolic Convolutional Neural Networks. ArXiv, abs/1905.10484.
 
 [50] Leimeister, M., & Wilson, B.J. (2018). Skip-gram word embeddings in hyperbolic space. ArXiv, abs/1809.01498.
 
 [51] Tifrea, A., Bécigneul, G., & Ganea, O. (2019). Poincaré GloVe: Hyperbolic Word Embeddings. ArXiv, abs/1810.06546.